In [1]:
# Loading the IMDB dataset
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/dados/anaconda3/lib/python3.8/site-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/dados/anaconda3/lib/python3.8/site-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'd

In [2]:
# Encoding the integer sequences into a binary matrix
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [3]:
# also vectorize your labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [4]:
import os
os.environ['PYTHONHASHSEED']=str(1)

import tensorflow as tf
import random
import numpy as np

def reset_random_seeds(SEED = 42):
  os.environ['PYTHONHASHSEED']=str(SEED)
  tf.random.set_seed(SEED)
  np.random.seed(SEED)
  random.seed(SEED)

In [5]:
from keras import models
from keras import layers

def create_and_train_model(hidden_layers=2,
                           hidden_units=16, 
                           loss_function='binary_crossentropy',
                           activation_function='relu'):
  
  model = models.Sequential()

  model.add(layers.Dense(hidden_units, activation=activation_function, input_shape=(10000,)))

  for i in range(hidden_layers - 1):
    model.add(layers.Dense(hidden_units, activation=activation_function))
 
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop',
                loss=loss_function,
                metrics=['accuracy'])

  model.fit(x_train,
            y_train,
            epochs=4,
            verbose=0,
            batch_size=512)

  results = model.evaluate(x_test, y_test)

  return "[hidden_layers=%d, hidden_units=%d, loss_function=%s, " \
          "activation_function=%s]\n => loss: %.4f - accuracy: %.4f\n" % (
          hidden_layers, hidden_units, loss_function, activation_function,
          results[0], results[1])

In [6]:
%%time

# You used two hidden layers. Try using one or three hidden layers, 
# and see how doing so affects validation and test accuracy.

# => best result: 2 hidden layers

for value in [1, 2, 3, 4]:
  res = create_and_train_model(hidden_layers=value)
  print(res)

782/782 [==============================] - 1s 584us/step - loss: 0.3036 - accuracy: 0.8776
[hidden_layers=1, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3036 - accuracy: 0.8776

782/782 [==============================] - 1s 576us/step - loss: 0.2860 - accuracy: 0.8868
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.2860 - accuracy: 0.8868

782/782 [==============================] - 1s 588us/step - loss: 0.3057 - accuracy: 0.8801
[hidden_layers=3, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3057 - accuracy: 0.8801

782/782 [==============================] - 1s 583us/step - loss: 0.3923 - accuracy: 0.8528
[hidden_layers=4, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3923 - accuracy: 0.8528

CPU times: user 55.4 s, sys: 5.07 s, total: 1min
Wall time: 21.9 s


In [7]:
%%time

# Try using layers with more hidden units or fewer hidden units: 
# 32 units, 64 units, and so on.

# => best result: 8 hidden units

for value in [4, 8, 16, 32]:
  res = create_and_train_model(hidden_units=value)
  print(res)

782/782 [==============================] - 1s 1ms/step - loss: 0.3040 - accuracy: 0.8877
[hidden_layers=2, hidden_units=4, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3040 - accuracy: 0.8877

782/782 [==============================] - 1s 577us/step - loss: 0.2816 - accuracy: 0.8908
[hidden_layers=2, hidden_units=8, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.2816 - accuracy: 0.8908

782/782 [==============================] - 1s 591us/step - loss: 0.3043 - accuracy: 0.8809
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3043 - accuracy: 0.8809

782/782 [==============================] - 1s 557us/step - loss: 0.3242 - accuracy: 0.8758
[hidden_layers=2, hidden_units=32, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3242 - accuracy: 0.8758

CPU times: user 49.8 s, sys: 4.4 s, total: 54.2 s
Wall time: 24.9 s


In [8]:
%%time

# Try using the mse loss function instead of binary_crossentropy.

# => best result: mse loss function

for value in ['binary_crossentropy', 'mse']:
  res = create_and_train_model(loss_function=value)
  print(res)

782/782 [==============================] - 1s 666us/step - loss: 0.3025 - accuracy: 0.8796
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.3025 - accuracy: 0.8796

782/782 [==============================] - 4s 2ms/step - loss: 0.0858 - accuracy: 0.8842
[hidden_layers=2, hidden_units=16, loss_function=mse, activation_function=relu]
 => loss: 0.0858 - accuracy: 0.8842

CPU times: user 25 s, sys: 10.7 s, total: 35.7 s
Wall time: 1min 25s


In [6]:
%%time

# Try using the tanh activation (an activation that was popular in 
# the early days of neural networks) instead of relu.

# => best result: relu activation function

for value in ['relu', 'tanh']:
  res = create_and_train_model(activation_function=value)
  print(res)

782/782 [==============================] - 1s 597us/step - loss: 0.2978 - accuracy: 0.8818
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_function=relu]
 => loss: 0.2978 - accuracy: 0.8818

782/782 [==============================] - 1s 593us/step - loss: 0.3150 - accuracy: 0.8808
[hidden_layers=2, hidden_units=16, loss_function=binary_crossentropy, activation_function=tanh]
 => loss: 0.3150 - accuracy: 0.8808

CPU times: user 32 s, sys: 3.07 s, total: 35.1 s
Wall time: 16.4 s
